In [17]:
import numpy as np
import cvxpy as cx

The thrusters numbered from 1-6 is as follows: <br>
T1: Front left vertical <br>
T2: Front right vertical <br>
T3: Back left vertical<br>
T4: Back right vertical <br>
T5: Side left horizontal <br>
T6: Side right horizontal <br>
The absolute distances to the side thruster are 0m in x and 0.185 in y. The absolute distances to the corner thrusters are 0.217m and 0.152m in y.

The Thruster Allocation Matrix is represented as:

$$
\begin{bmatrix}
F_x \\
F_z \\
M_{roll} \\
M_{pitch} \\
M_{yaw} \\
\end{bmatrix}
=
\begin{bmatrix}
0 & 0 & 0 & 0 & 1 & 1 \\
1 & 1 & 1 & 1 & 0 & 0 \\
-0.217 & 0.217 & -0.217 & 0.217 & 0 & 0 \\
0.152 & 0.152 & -0.152 & -0.152 & 0 & 0 \\
-0.189 & 0.189 & 0.189 & -0.189 & -0.185 & 0.185 \\
\end{bmatrix}
\begin{bmatrix}
T_1 \\
T_2 \\
T_3 \\
T_4 \\
T_5 \\
T_6 \\
\end{bmatrix}
$$

Where $F_x$, $F_z$ are the forces in the x and z directions, $M_{roll}$, $M_{pitch}$, and $M_{yaw}$ are the moments about the respective axes, and $T_1$ to $T_6$ are the thruster outputs.


In [106]:
T_alloc = np.array([
    [0, 0, 0, 0, 1, 1],  # F_x
    [1, 1, 1, 1, 0, 0],  # F_z
    [-0.217, 0.217, -0.217, 0.217, 0, 0],  # M_roll
    [0.152, 0.152, -0.152, -0.152, 0, 0],  # M_pitch
    [0, 0, 0, 0, -0.185, 0.185]  # M_yaw
]) # thruster allocation matrix

In [107]:
T_alloc_pinv = np.round(np.linalg.pinv(T_alloc), 3) # inverse of thruster allocation matrix
T_alloc_pinv

array([[ 0.   ,  0.25 , -1.152,  1.645,  0.   ],
       [ 0.   ,  0.25 ,  1.152,  1.645,  0.   ],
       [ 0.   ,  0.25 , -1.152, -1.645,  0.   ],
       [ 0.   ,  0.25 ,  1.152, -1.645,  0.   ],
       [ 0.5  , -0.   ,  0.   , -0.   , -2.703],
       [ 0.5  , -0.   ,  0.   , -0.   ,  2.703]])

In [108]:
T_alloc_pinv = np.linalg.pinv(T_alloc) # inverse of thruster allocation matrix

The pseudo-inverse of the Thruster Allocation Matrix is represented as:

$$
\begin{bmatrix}
T_1 \\
T_2 \\
T_3 \\
T_4 \\
T_5 \\
T_6 \\
\end{bmatrix}
=
\begin{bmatrix}
0 & 0.25 & -1.152 & 1.645 & 0 \\
0 & 0.25 & 1.152 & 1.645 & 0 \\
0 & 0.25 & -1.152 & -1.645 & 0 \\
0 & 0.25 & 1.152 & -1.645 & 0 \\
0.5 & 0 & 0 & 0 & -2.703 \\
0.5 & 0 & 0 & 0 & 2.703 \\
\end{bmatrix}
\begin{bmatrix}
F_{x} \\
F_{z} \\
M_{\text{{roll}}} \\
M_{\text{{pitch}}} \\
M_{\text{{yaw}}} \\
\end{bmatrix}
$$

Where $T_1$ to $T_6$ are the thruster outputs, and $F_{x}$, $F_{z}$ are the forces in the x and z directions, $M_{\text{{roll}}}$, $M_{\text{{pitch}}}$, and $M_{\text{{yaw}}}$ are the moments about the respective axes.



In [87]:
F_x = 1
F_z = 0
M_roll = 1
M_pitch = 0
M_yaw = 0

desired_force_torque = np.array([F_x, F_z, M_roll, M_pitch, M_yaw])
initial_thruster_values = T_alloc_pinv @ desired_force_torque

thruster_values = cx.Variable(6) # 6 thrusters
objective = cx.Minimize(cx.norm(T_alloc @ thruster_values - desired_force_torque, "inf"))
constraints = [thruster_values >= -1, thruster_values <= 1]
problem = cx.Problem(objective, constraints)
problem.solve()
optimized_thruster_values = thruster_values.value
print(initial_thruster_values)
print(optimized_thruster_values)

[-1.15207373  1.15207373 -1.15207373  1.15207373  0.5         0.5       ]
[-1.          1.         -1.          1.          0.49803818  0.49803818]


In [89]:
actual_force_torque = T_alloc @ optimized_thruster_values
error = np.square(desired_force_torque - actual_force_torque)
print(np.round(error, 3))

[0.    0.    0.017 0.    0.   ]
